In [ ]:
# https://www.spotrac.com/mls/transactions/trade/
# 
# article > table > tbody > tr:child(1) > h3 -- date - need to figure out year (could use url year )
# article > table > tbody > ts:child(2) > td > span.tradelabel -- team (remove "acquires")
# article > table > tbody > ts:child(2) > td > span.tradeplayer -- asset received (can be multiple of these)
# 
# 
# 
# result: JSON of trades
# range: 2020 to 2022

In [194]:
import requests
from bs4 import BeautifulSoup

seasons = range(2018, 2020)

composite_list = {}

for year in seasons:
    print(f"getting spotrac transactions for {year}")
    response = requests.get(f'https://www.spotrac.com/mls/transactions/{year}/trade/')
    html = response.text
    parsed_html = BeautifulSoup(html, 'html.parser')

    result = []
    for transaction in parsed_html.body.find_all('article'):
        base_table = transaction.table
        if (base_table == None):
            continue

        rows = base_table.find_all('tr')
        base = {'year': year, 'date' : rows[0].td.h3.text, 'transactions': []}
        for club in rows[1].find_all('td'):
            club_name = club.find('span', attrs={"class":"tradelabel"}).text.replace(' acquires', '')
            raw_assets_acquired = club.find_all('span', attrs={"class":"tradeplayer"})
            assets_acquired = []
            for asset in raw_assets_acquired:
                assets_acquired.append(asset.text)
            base['transactions'].append({ 'club': club_name, 'assets_acquired': assets_acquired })

        result.append(base)
    print(f"parsed out {len(result)} items")
    composite_list[year] = result    
    
composite_list

getting spotrac transactions for 2018
parsed out 42 items
getting spotrac transactions for 2019
parsed out 42 items


{2018: [{'year': 2018,
   'date': 'Dec 28',
   'transactions': [{'club': 'Chicago',
     'assets_acquired': ['2020 4th round pick ']},
    {'club': 'Seattle', 'assets_acquired': ['Jonathan Campbell (D)']}]},
  {'year': 2018,
   'date': 'Dec 27',
   'transactions': [{'club': 'Columbus',
     'assets_acquired': ['Joe Bendik (GK)']},
    {'club': 'Orlando City',
     'assets_acquired': ['$50k Targeted Allocation Money']}]},
  {'year': 2018,
   'date': 'Dec 19',
   'transactions': [{'club': 'D.C.',
     'assets_acquired': ['Earl Edwards, Jr. (GK)']},
    {'club': 'Orlando City',
     'assets_acquired': ['2019 2nd round pick (#38)']}]},
  {'year': 2018,
   'date': 'Dec 19',
   'transactions': [{'club': 'Philadelphia',
     'assets_acquired': ['$150k General Allocation Money (2019), $50k Targeted and $100k General Allocation Money (2020)']},
    {'club': 'Colorado', 'assets_acquired': ['Keegan Rosenberry (D)']}]},
  {'year': 2018,
   'date': 'Dec 18',
   'transactions': [{'club': 'Kansas Cit

In [195]:
import json

# with open("./mls_trades_2018.json", "w") as outfile:
#     outfile.write(json.dumps(composite_list, indent = 4))

In [196]:
with open('./mls_trades.json', 'r') as openfile:
    # Reading from json file
    corrected_file = json.load(openfile)
    
corrected_file

{'2020': [{'year': 2020,
   'date': 'Dec 29',
   'transactions': [{'club': 'New York City FC',
     'assets_acquired': ['$250000 2021 GAM',
      '$250000 2022 GAM',
      '$100000 2022 GAM (performance bonus)']},
    {'club': 'FC Cincinnati', 'assets_acquired': ['Ronald Matarrita']}]},
  {'year': 2020,
   'date': 'Dec 23',
   'transactions': [{'club': 'New York Red Bulls',
     'assets_acquired': ['$50000 2021 GAM', '2021 3rd round pick']},
    {'club': 'Inter Miami', 'assets_acquired': ['Patrick Seagrist']}]},
  {'year': 2020,
   'date': 'Dec 18',
   'transactions': [{'club': 'Chicago',
     'assets_acquired': ['2022 4th round pick (swap)']},
    {'club': 'Charlotte FC',
     'assets_acquired': ['Brandt Bronico', '2022 4th round pick (swap)']}]},
  {'year': 2020,
   'date': 'Dec 17',
   'transactions': [{'club': 'New York City FC',
     'assets_acquired': ['$500000 2021 GAM',
      '$250000 2022 GAM',
      '$500000 2022 GAM (performance bonus)']},
    {'club': 'Austin FC', 'assets_a

In [198]:
with open('./mls_trades_2018.json', 'r') as openfile:
    # Reading from json file
    corrected_file_2018 = json.load(openfile)
    
corrected_file_2018

{'2018': [{'year': 2018,
   'date': 'Dec 28',
   'transactions': [{'club': 'Chicago',
     'assets_acquired': ['2020 4th round pick']},
    {'club': 'Seattle Sounders', 'assets_acquired': ['Jonathan Campbell']}]},
  {'year': 2018,
   'date': 'Dec 27',
   'transactions': [{'club': 'Columbus Crew',
     'assets_acquired': ['Joe Bendik (GK)']},
    {'club': 'Orlando City', 'assets_acquired': ['$50000 2019 TAM']}]},
  {'year': 2018,
   'date': 'Dec 19',
   'transactions': [{'club': 'D.C. United',
     'assets_acquired': ['Earl Edwards, Jr.']},
    {'club': 'Orlando City',
     'assets_acquired': ['2019 2nd round pick (#38)']}]},
  {'year': 2018,
   'date': 'Dec 19',
   'transactions': [{'club': 'Philadelphia',
     'assets_acquired': ['$150000 2019 GAM',
      '$50000 2020 TAM',
      '$100000 2020 GAM']},
    {'club': 'Colorado Rapids', 'assets_acquired': ['Keegan Rosenberry']}]},
  {'year': 2018,
   'date': 'Dec 18',
   'transactions': [{'club': 'Kansas City',
     'assets_acquired': ['K

In [199]:
records = []
for arr in corrected_file.values():
    records += arr
    
for arr in corrected_file_2018.values():
    records += arr
records

[{'year': 2020,
  'date': 'Dec 29',
  'transactions': [{'club': 'New York City FC',
    'assets_acquired': ['$250000 2021 GAM',
     '$250000 2022 GAM',
     '$100000 2022 GAM (performance bonus)']},
   {'club': 'FC Cincinnati', 'assets_acquired': ['Ronald Matarrita']}]},
 {'year': 2020,
  'date': 'Dec 23',
  'transactions': [{'club': 'New York Red Bulls',
    'assets_acquired': ['$50000 2021 GAM', '2021 3rd round pick']},
   {'club': 'Inter Miami', 'assets_acquired': ['Patrick Seagrist']}]},
 {'year': 2020,
  'date': 'Dec 18',
  'transactions': [{'club': 'Chicago',
    'assets_acquired': ['2022 4th round pick (swap)']},
   {'club': 'Charlotte FC',
    'assets_acquired': ['Brandt Bronico', '2022 4th round pick (swap)']}]},
 {'year': 2020,
  'date': 'Dec 17',
  'transactions': [{'club': 'New York City FC',
    'assets_acquired': ['$500000 2021 GAM',
     '$250000 2022 GAM',
     '$500000 2022 GAM (performance bonus)']},
   {'club': 'Austin FC', 'assets_acquired': ['Alexander Ring']}]},


In [200]:
base_df = pd.DataFrame(records)
base_df['trade_id'] = range(0, len(base_df))
base_df['raw_trade_date'] = base_df.agg('{0[date]}, {0[year]}'.format, axis=1)
base_df['trade_date'] = pd.to_datetime(base_df['raw_trade_date'], errors='coerce')
base_df = base_df[['trade_id', 'trade_date', 'transactions']]
base_df

,trade_id,trade_date,transactions
0,0,2020-12-29,"[{'club': 'New York City FC', 'assets_acquired..."
1,1,2020-12-23,"[{'club': 'New York Red Bulls', 'assets_acquir..."
2,2,2020-12-18,"[{'club': 'Chicago', 'assets_acquired': ['2022..."
3,3,2020-12-17,"[{'club': 'New York City FC', 'assets_acquired..."
4,4,2020-12-17,"[{'club': 'Houston', 'assets_acquired': ['Fafa..."
...,...,...,...
187,187,2019-01-24,"[{'club': 'Chicago', 'assets_acquired': ['Aman..."
188,188,2019-01-23,"[{'club': 'Toronto FC', 'assets_acquired': ['$..."
189,189,2019-01-23,"[{'club': 'New England', 'assets_acquired': ['..."
190,190,2019-01-11,"[{'club': 'Houston', 'assets_acquired': ['2019..."


In [201]:
transaction_expl = json.loads(base_df.explode('transactions').to_json(orient="records"))
transaction_expl_flat = pd.json_normalize(transaction_expl)

transaction_expl_flat

,trade_id,trade_date,transactions.club,transactions.assets_acquired
0,0,1609200000000,New York City FC,"[$250000 2021 GAM, $250000 2022 GAM, $100000 2..."
1,0,1609200000000,FC Cincinnati,[Ronald Matarrita]
2,1,1608681600000,New York Red Bulls,"[$50000 2021 GAM, 2021 3rd round pick]"
3,1,1608681600000,Inter Miami,[Patrick Seagrist]
4,2,1608249600000,Chicago,[2022 4th round pick (swap)]
...,...,...,...,...
380,189,1548201600000,Portland,[Claude Dielna]
381,190,1547164800000,Houston,[2019 2nd round pick (#33)]
382,190,1547164800000,D.C. United,[Chris Seitz (GK)]
383,191,1547164800000,Houston,[Marlon Hairston]


In [202]:
compiled_assets = pd.DataFrame({ 'trade_id' : [], 'transactions.club' : [], 'assets': []})

for i, row in transaction_expl_flat.iterrows():
    new_assets = []
    raw_assets = row['transactions.assets_acquired']
    
    for asset in raw_assets:
        base_asset_record = {'asset_type': None, 'value' : None}
        if ('GAM' in asset):
            base_asset_record['asset_type'] = 'GAM'
        if ('TAM' in asset):
            base_asset_record['asset_type'] = 'TAM'
        elif (' pick' in asset or 'Draft' in asset):
            base_asset_record['asset_type'] = 'Draft Pick'
        elif ('International' in asset):
            base_asset_record['asset_type'] = 'International Slot'
        elif ('sell-on' in asset or 'sellon' in asset or 'sell on' in asset):
            base_asset_record['asset_type'] = 'Sell-On Clause'
        else:
            base_asset_record['asset_type'] = 'Player'
        
        base_asset_record['value'] = asset
        new_assets.append(base_asset_record)
        
    compiled_assets = compiled_assets.append({ 'trade_id': row.trade_id, 'transactions.club': row['transactions.club'], 'assets': new_assets }, ignore_index=True)

compiled_assets  

,trade_id,transactions.club,assets
0,0.0,New York City FC,"[{'asset_type': 'Player', 'value': '$250000 20..."
1,0.0,FC Cincinnati,"[{'asset_type': 'Player', 'value': 'Ronald Mat..."
2,1.0,New York Red Bulls,"[{'asset_type': 'Player', 'value': '$50000 202..."
3,1.0,Inter Miami,"[{'asset_type': 'Player', 'value': 'Patrick Se..."
4,2.0,Chicago,"[{'asset_type': 'Draft Pick', 'value': '2022 4..."
...,...,...,...
380,189.0,Portland,"[{'asset_type': 'Player', 'value': 'Claude Die..."
381,190.0,Houston,"[{'asset_type': 'Draft Pick', 'value': '2019 2..."
382,190.0,D.C. United,"[{'asset_type': 'Player', 'value': 'Chris Seit..."
383,191.0,Houston,"[{'asset_type': 'Player', 'value': 'Marlon Hai..."


In [203]:
composite_transactions = pd.merge(transaction_expl_flat, compiled_assets, left_on=['trade_id','transactions.club'], right_on=['trade_id','transactions.club'])
composite_transactions

,trade_id,trade_date,transactions.club,transactions.assets_acquired,assets
0,0,1609200000000,New York City FC,"[$250000 2021 GAM, $250000 2022 GAM, $100000 2...","[{'asset_type': 'Player', 'value': '$250000 20..."
1,0,1609200000000,FC Cincinnati,[Ronald Matarrita],"[{'asset_type': 'Player', 'value': 'Ronald Mat..."
2,1,1608681600000,New York Red Bulls,"[$50000 2021 GAM, 2021 3rd round pick]","[{'asset_type': 'Player', 'value': '$50000 202..."
3,1,1608681600000,Inter Miami,[Patrick Seagrist],"[{'asset_type': 'Player', 'value': 'Patrick Se..."
4,2,1608249600000,Chicago,[2022 4th round pick (swap)],"[{'asset_type': 'Draft Pick', 'value': '2022 4..."
...,...,...,...,...,...
380,189,1548201600000,Portland,[Claude Dielna],"[{'asset_type': 'Player', 'value': 'Claude Die..."
381,190,1547164800000,Houston,[2019 2nd round pick (#33)],"[{'asset_type': 'Draft Pick', 'value': '2019 2..."
382,190,1547164800000,D.C. United,[Chris Seitz (GK)],"[{'asset_type': 'Player', 'value': 'Chris Seit..."
383,191,1547164800000,Houston,[Marlon Hairston],"[{'asset_type': 'Player', 'value': 'Marlon Hai..."


In [204]:
asset_expl = json.loads(composite_transactions.explode('assets').to_json(orient="records"))
asset_expl_flat = pd.json_normalize(asset_expl)
asset_expl_flat = asset_expl_flat[['trade_id', 'trade_date', 'transactions.club', 'assets.asset_type', 'assets.value']]
asset_expl_flat.columns = ['trade_id', 'trade_date', 'club','acquired_asset_type', 'acquired_asset_value']
asset_expl_flat.trade_date = pd.to_datetime(asset_expl_flat.trade_date, unit='ms')
asset_expl_flat['acquired_asset_id'] = range(0, len(asset_expl_flat))
asset_expl_flat

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id
0,0,2020-12-29,New York City FC,Player,$250000 2021 GAM,0
1,0,2020-12-29,New York City FC,Player,$250000 2022 GAM,1
2,0,2020-12-29,New York City FC,Player,$100000 2022 GAM (performance bonus),2
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3
4,1,2020-12-23,New York Red Bulls,Player,$50000 2021 GAM,4
...,...,...,...,...,...,...
522,190,2019-01-11,Houston,Draft Pick,2019 2nd round pick (#33),522
523,190,2019-01-11,D.C. United,Player,Chris Seitz (GK),523
524,191,2019-01-11,Houston,Player,Marlon Hairston,524
525,191,2019-01-11,Colorado Rapids,Player,$125000 2019 GAM,525


In [214]:
import numpy as np 

asset_expl_flat['gam_value'] = asset_expl_flat.acquired_asset_value.str.extract('(\d+) \d{4} GAM')
asset_expl_flat['gam_value'] = np.where(asset_expl_flat.gam_value.isna(), 0, asset_expl_flat.gam_value)

asset_expl_flat['tam_value'] = asset_expl_flat.acquired_asset_value.str.extract('(\d+) \d{4} TAM')
asset_expl_flat['tam_value'] = np.where(asset_expl_flat.tam_value.isna(), 0, asset_expl_flat.tam_value)

asset_expl_flat['asset_year'] = asset_expl_flat.acquired_asset_value.str.extract('\d+ (\d{4}) GAM')
asset_expl_flat['asset_year'] = np.where(asset_expl_flat.asset_year.isna(), asset_expl_flat.trade_date.dt.year, asset_expl_flat.asset_year)

asset_expl_flat.asset_year = asset_expl_flat.asset_year.astype(int)
asset_expl_flat.gam_value = asset_expl_flat.gam_value.astype(int)
asset_expl_flat.tam_value = asset_expl_flat.tam_value.astype(int)

asset_expl_flat

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year,tam_value
0,0,2020-12-29,New York City FC,Player,$250000 2021 GAM,0,250000,2021,0
1,0,2020-12-29,New York City FC,Player,$250000 2022 GAM,1,250000,2022,0
2,0,2020-12-29,New York City FC,Player,$100000 2022 GAM (performance bonus),2,100000,2022,0
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3,0,2020,0
4,1,2020-12-23,New York Red Bulls,Player,$50000 2021 GAM,4,50000,2021,0
...,...,...,...,...,...,...,...,...,...
522,190,2019-01-11,Houston,Draft Pick,2019 2nd round pick (#33),522,0,2019,0
523,190,2019-01-11,D.C. United,Player,Chris Seitz (GK),523,0,2019,0
524,191,2019-01-11,Houston,Player,Marlon Hairston,524,0,2019,0
525,191,2019-01-11,Colorado Rapids,Player,$125000 2019 GAM,525,125000,2019,0


In [215]:
# save state
asset_expl_flat.to_csv('./allocation_money/mls_trade_db.csv', index=False)

In [219]:
asset_expl_flat.club.unique()

array(['New York City FC', 'FC Cincinnati', 'New York Red Bulls',
       'Inter Miami', 'Chicago', 'Charlotte FC', 'Austin FC', 'Houston',
       'FC Dallas', 'CF Montreal', 'Portland', 'LAFC', 'Atlanta United',
       'San Jose', 'Columbus Crew', 'Vancouver Whitecaps', 'Minnesota',
       'Seattle Sounders', 'Nashville', 'D.C. United', 'Colorado Rapids',
       'New England', 'Philadelphia', 'Kansas City', 'LA Galaxy',
       'Real Salt Lake', 'Orlando City', 'Toronto FC'], dtype=object)

In [220]:
# create team GAM table for a year
# group each trade's rows
# for each trade
# - add sum of GAM values for acquired teams (IE: their rows) to team table
# - subtract for non-team rows still in the trade subtract GAM value from team table
asset_expl_flat

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year,tam_value
0,0,2020-12-29,New York City FC,Player,$250000 2021 GAM,0,250000,2021,0
1,0,2020-12-29,New York City FC,Player,$250000 2022 GAM,1,250000,2022,0
2,0,2020-12-29,New York City FC,Player,$100000 2022 GAM (performance bonus),2,100000,2022,0
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3,0,2020,0
4,1,2020-12-23,New York Red Bulls,Player,$50000 2021 GAM,4,50000,2021,0
...,...,...,...,...,...,...,...,...,...
522,190,2019-01-11,Houston,Draft Pick,2019 2nd round pick (#33),522,0,2019,0
523,190,2019-01-11,D.C. United,Player,Chris Seitz (GK),523,0,2019,0
524,191,2019-01-11,Houston,Player,Marlon Hairston,524,0,2019,0
525,191,2019-01-11,Colorado Rapids,Player,$125000 2019 GAM,525,125000,2019,0


In [221]:
am_table = {  }

available_asset_years = range(int(asset_expl_flat.asset_year.min()), int(asset_expl_flat.asset_year.max()) + 1)
for year in available_asset_years:
    am_table[str(year)] = {}
    for club in asset_expl_flat.club.unique():
        am_table[str(year)][club] = {}
        am_table[str(year)][club]['GAM'] = 0
        am_table[str(year)][club]['TAM'] = 0
        

    
# there's a three team trade in here to look out for, ignore it
grouped_trades = asset_expl_flat.groupby('trade_id')
for trade_id, trade in grouped_trades:
    gam_groups = trade.groupby('club')

    if (len(gam_groups) == 2):
        for club, rows in gam_groups:
            for year in available_asset_years:
                selected_asset_year = str(year)
                used_assets = rows.acquired_asset_id.tolist()
                
                gam_add = rows[rows.asset_year == year].gam_value.sum()
                gam_remove = trade[(trade.asset_year == year) & ~(trade.acquired_asset_id.isin(used_assets))].gam_value.sum()
                
                print(f'{selected_asset_year}: Adding {gam_add} GAM to {club}')
                am_table[selected_asset_year][club]['GAM'] += gam_add
                print(f'{selected_asset_year}: Subtracting {gam_remove} GAM from {club}')
                am_table[selected_asset_year][club]['GAM'] -= gam_remove
                
                tam_add = rows[rows.asset_year == year].tam_value.sum()
                tam_remove = trade[(trade.asset_year == year) & ~(trade.acquired_asset_id.isin(used_assets))].tam_value.sum()
                
                print(f'{selected_asset_year}: Adding {tam_add} TAM to {club}')
                am_table[selected_asset_year][club]['TAM'] += tam_add
                print(f'{selected_asset_year}: Subtracting {tam_remove} TAM from {club}')
                am_table[selected_asset_year][club]['TAM'] -= tam_remove

    else:
        print(f'trade_id {trade_id} had weird number of teams: {len(gam_groups)}')
        
am_table  

2018: Adding 0 GAM to FC Cincinnati
2018: Subtracting 0 GAM from FC Cincinnati
2018: Adding 0 TAM to FC Cincinnati
2018: Subtracting 0 TAM from FC Cincinnati
2019: Adding 0 GAM to FC Cincinnati
2019: Subtracting 0 GAM from FC Cincinnati
2019: Adding 0 TAM to FC Cincinnati
2019: Subtracting 0 TAM from FC Cincinnati
2020: Adding 0 GAM to FC Cincinnati
2020: Subtracting 0 GAM from FC Cincinnati
2020: Adding 0 TAM to FC Cincinnati
2020: Subtracting 0 TAM from FC Cincinnati
2021: Adding 0 GAM to FC Cincinnati
2021: Subtracting 250000 GAM from FC Cincinnati
2021: Adding 0 TAM to FC Cincinnati
2021: Subtracting 0 TAM from FC Cincinnati
2022: Adding 0 GAM to FC Cincinnati
2022: Subtracting 350000 GAM from FC Cincinnati
2022: Adding 0 TAM to FC Cincinnati
2022: Subtracting 0 TAM from FC Cincinnati
2023: Adding 0 GAM to FC Cincinnati
2023: Subtracting 0 GAM from FC Cincinnati
2023: Adding 0 TAM to FC Cincinnati
2023: Subtracting 0 TAM from FC Cincinnati
2024: Adding 0 GAM to FC Cincinnati
2024: 

2018: Adding 0 GAM to Portland
2018: Subtracting 0 GAM from Portland
2018: Adding 0 TAM to Portland
2018: Subtracting 0 TAM from Portland
2019: Adding 0 GAM to Portland
2019: Subtracting 0 GAM from Portland
2019: Adding 0 TAM to Portland
2019: Subtracting 0 TAM from Portland
2020: Adding 0 GAM to Portland
2020: Subtracting 0 GAM from Portland
2020: Adding 0 TAM to Portland
2020: Subtracting 0 TAM from Portland
2021: Adding 300000 GAM to Portland
2021: Subtracting 0 GAM from Portland
2021: Adding 0 TAM to Portland
2021: Subtracting 0 TAM from Portland
2022: Adding 0 GAM to Portland
2022: Subtracting 0 GAM from Portland
2022: Adding 0 TAM to Portland
2022: Subtracting 0 TAM from Portland
2023: Adding 0 GAM to Portland
2023: Subtracting 0 GAM from Portland
2023: Adding 0 TAM to Portland
2023: Subtracting 0 TAM from Portland
2024: Adding 0 GAM to Portland
2024: Subtracting 0 GAM from Portland
2024: Adding 0 TAM to Portland
2024: Subtracting 0 TAM from Portland
2025: Adding 0 GAM to Portlan

2022: Adding 0 GAM to Nashville
2022: Subtracting 0 GAM from Nashville
2022: Adding 0 TAM to Nashville
2022: Subtracting 0 TAM from Nashville
2023: Adding 0 GAM to Nashville
2023: Subtracting 0 GAM from Nashville
2023: Adding 0 TAM to Nashville
2023: Subtracting 0 TAM from Nashville
2024: Adding 0 GAM to Nashville
2024: Subtracting 0 GAM from Nashville
2024: Adding 0 TAM to Nashville
2024: Subtracting 0 TAM from Nashville
2025: Adding 0 GAM to Nashville
2025: Subtracting 0 GAM from Nashville
2025: Adding 0 TAM to Nashville
2025: Subtracting 0 TAM from Nashville
2018: Adding 0 GAM to Seattle Sounders
2018: Subtracting 0 GAM from Seattle Sounders
2018: Adding 0 TAM to Seattle Sounders
2018: Subtracting 0 TAM from Seattle Sounders
2019: Adding 0 GAM to Seattle Sounders
2019: Subtracting 0 GAM from Seattle Sounders
2019: Adding 0 TAM to Seattle Sounders
2019: Subtracting 0 TAM from Seattle Sounders
2020: Adding 0 GAM to Seattle Sounders
2020: Subtracting 250000 GAM from Seattle Sounders
20

2021: Adding 0 GAM to Minnesota
2021: Subtracting 0 GAM from Minnesota
2021: Adding 0 TAM to Minnesota
2021: Subtracting 0 TAM from Minnesota
2022: Adding 0 GAM to Minnesota
2022: Subtracting 0 GAM from Minnesota
2022: Adding 0 TAM to Minnesota
2022: Subtracting 0 TAM from Minnesota
2023: Adding 0 GAM to Minnesota
2023: Subtracting 0 GAM from Minnesota
2023: Adding 0 TAM to Minnesota
2023: Subtracting 0 TAM from Minnesota
2024: Adding 0 GAM to Minnesota
2024: Subtracting 0 GAM from Minnesota
2024: Adding 0 TAM to Minnesota
2024: Subtracting 0 TAM from Minnesota
2025: Adding 0 GAM to Minnesota
2025: Subtracting 0 GAM from Minnesota
2025: Adding 0 TAM to Minnesota
2025: Subtracting 0 TAM from Minnesota
2018: Adding 0 GAM to Nashville
2018: Subtracting 0 GAM from Nashville
2018: Adding 0 TAM to Nashville
2018: Subtracting 0 TAM from Nashville
2019: Adding 0 GAM to Nashville
2019: Subtracting 0 GAM from Nashville
2019: Adding 0 TAM to Nashville
2019: Subtracting 0 TAM from Nashville
2020: 

2024: Subtracting 0 TAM from Columbus Crew
2025: Adding 0 GAM to Columbus Crew
2025: Subtracting 0 GAM from Columbus Crew
2025: Adding 0 TAM to Columbus Crew
2025: Subtracting 0 TAM from Columbus Crew
2018: Adding 0 GAM to Minnesota
2018: Subtracting 0 GAM from Minnesota
2018: Adding 0 TAM to Minnesota
2018: Subtracting 0 TAM from Minnesota
2019: Adding 0 GAM to Minnesota
2019: Subtracting 0 GAM from Minnesota
2019: Adding 0 TAM to Minnesota
2019: Subtracting 0 TAM from Minnesota
2020: Adding 0 GAM to Minnesota
2020: Subtracting 0 GAM from Minnesota
2020: Adding 0 TAM to Minnesota
2020: Subtracting 0 TAM from Minnesota
2021: Adding 0 GAM to Minnesota
2021: Subtracting 0 GAM from Minnesota
2021: Adding 0 TAM to Minnesota
2021: Subtracting 0 TAM from Minnesota
2022: Adding 0 GAM to Minnesota
2022: Subtracting 0 GAM from Minnesota
2022: Adding 0 TAM to Minnesota
2022: Subtracting 0 TAM from Minnesota
2023: Adding 0 GAM to Minnesota
2023: Subtracting 0 GAM from Minnesota
2023: Adding 0 TAM

...

2020: Adding 0 GAM to Inter Miami
2020: Subtracting 50000 GAM from Inter Miami
2020: Adding 0 TAM to Inter Miami
2020: Subtracting 0 TAM from Inter Miami
2021: Adding 0 GAM to Inter Miami
2021: Subtracting 0 GAM from Inter Miami
2021: Adding 0 TAM to Inter Miami
2021: Subtracting 0 TAM from Inter Miami
2022: Adding 0 GAM to Inter Miami
2022: Subtracting 0 GAM from Inter Miami
2022: Adding 0 TAM to Inter Miami
2022: Subtracting 0 TAM from Inter Miami
2023: Adding 0 GAM to Inter Miami
2023: Subtracting 0 GAM from Inter Miami
2023: Adding 0 TAM to Inter Miami
2023: Subtracting 0 TAM from Inter Miami
2024: Adding 0 GAM to Inter Miami
2024: Subtracting 0 GAM from Inter Miami
2024: Adding 0 TAM to Inter Miami
2024: Subtracting 0 TAM from Inter Miami
2025: Adding 0 GAM to Inter Miami
2025: Subtracting 0 GAM from Inter Miami
2025: Adding 0 TAM to Inter Miami
2025: Subtracting 0 TAM from Inter Miami
2018: Adding 0 GAM to Inter Miami
2018: Subtracting 0 GAM from Inter Miami
2018: Adding 0 TAM to

2022: Adding 0 GAM to New York City FC
2022: Subtracting 0 GAM from New York City FC
2022: Adding 0 TAM to New York City FC
2022: Subtracting 0 TAM from New York City FC
2023: Adding 0 GAM to New York City FC
2023: Subtracting 0 GAM from New York City FC
2023: Adding 0 TAM to New York City FC
2023: Subtracting 0 TAM from New York City FC
2024: Adding 0 GAM to New York City FC
2024: Subtracting 0 GAM from New York City FC
2024: Adding 0 TAM to New York City FC
2024: Subtracting 0 TAM from New York City FC
2025: Adding 0 GAM to New York City FC
2025: Subtracting 0 GAM from New York City FC
2025: Adding 0 TAM to New York City FC
2025: Subtracting 0 TAM from New York City FC
2018: Adding 0 GAM to Columbus Crew
2018: Subtracting 0 GAM from Columbus Crew
2018: Adding 0 TAM to Columbus Crew
2018: Subtracting 0 TAM from Columbus Crew
2019: Adding 0 GAM to Columbus Crew
2019: Subtracting 0 GAM from Columbus Crew
2019: Adding 0 TAM to Columbus Crew
2019: Subtracting 0 TAM from Columbus Crew
2020

2024: Adding 0 GAM to Colorado Rapids
2024: Subtracting 0 GAM from Colorado Rapids
2024: Adding 0 TAM to Colorado Rapids
2024: Subtracting 0 TAM from Colorado Rapids
2025: Adding 0 GAM to Colorado Rapids
2025: Subtracting 0 GAM from Colorado Rapids
2025: Adding 0 TAM to Colorado Rapids
2025: Subtracting 0 TAM from Colorado Rapids
2018: Adding 0 GAM to Kansas City
2018: Subtracting 0 GAM from Kansas City
2018: Adding 0 TAM to Kansas City
2018: Subtracting 0 TAM from Kansas City
2019: Adding 0 GAM to Kansas City
2019: Subtracting 0 GAM from Kansas City
2019: Adding 0 TAM to Kansas City
2019: Subtracting 50000 TAM from Kansas City
2020: Adding 0 GAM to Kansas City
2020: Subtracting 0 GAM from Kansas City
2020: Adding 0 TAM to Kansas City
2020: Subtracting 0 TAM from Kansas City
2021: Adding 0 GAM to Kansas City
2021: Subtracting 0 GAM from Kansas City
2021: Adding 0 TAM to Kansas City
2021: Subtracting 0 TAM from Kansas City
2022: Adding 0 GAM to Kansas City
2022: Subtracting 0 GAM from K

2020: Adding 0 GAM to Kansas City
2020: Subtracting 0 GAM from Kansas City
2020: Adding 0 TAM to Kansas City
2020: Subtracting 0 TAM from Kansas City
2021: Adding 0 GAM to Kansas City
2021: Subtracting 0 GAM from Kansas City
2021: Adding 0 TAM to Kansas City
2021: Subtracting 0 TAM from Kansas City
2022: Adding 0 GAM to Kansas City
2022: Subtracting 0 GAM from Kansas City
2022: Adding 0 TAM to Kansas City
2022: Subtracting 0 TAM from Kansas City
2023: Adding 0 GAM to Kansas City
2023: Subtracting 0 GAM from Kansas City
2023: Adding 0 TAM to Kansas City
2023: Subtracting 0 TAM from Kansas City
2024: Adding 0 GAM to Kansas City
2024: Subtracting 0 GAM from Kansas City
2024: Adding 0 TAM to Kansas City
2024: Subtracting 0 TAM from Kansas City
2025: Adding 0 GAM to Kansas City
2025: Subtracting 0 GAM from Kansas City
2025: Adding 0 TAM to Kansas City
2025: Subtracting 0 TAM from Kansas City
2018: Adding 0 GAM to Minnesota
2018: Subtracting 0 GAM from Minnesota
2018: Adding 0 TAM to Minneso

{'2018': {'New York City FC': {'GAM': 0, 'TAM': 0},
  'FC Cincinnati': {'GAM': -135000, 'TAM': -1050000},
  'New York Red Bulls': {'GAM': -225000, 'TAM': -150000},
  'Inter Miami': {'GAM': 0, 'TAM': 0},
  'Chicago': {'GAM': 1125000, 'TAM': -200000},
  'Charlotte FC': {'GAM': 0, 'TAM': 0},
  'Austin FC': {'GAM': 0, 'TAM': 0},
  'Houston': {'GAM': 0, 'TAM': -100000},
  'FC Dallas': {'GAM': 0, 'TAM': 225000},
  'CF Montreal': {'GAM': 0, 'TAM': -75000},
  'Portland': {'GAM': 0, 'TAM': 500000},
  'LAFC': {'GAM': 0, 'TAM': 0},
  'Atlanta United': {'GAM': 300000, 'TAM': 450000},
  'San Jose': {'GAM': 135000, 'TAM': 0},
  'Columbus Crew': {'GAM': 300000, 'TAM': 150000},
  'Vancouver Whitecaps': {'GAM': 225000, 'TAM': 200000},
  'Minnesota': {'GAM': 125000, 'TAM': 100000},
  'Seattle Sounders': {'GAM': 50000, 'TAM': 0},
  'Nashville': {'GAM': 0, 'TAM': 0},
  'D.C. United': {'GAM': -350000, 'TAM': -50000},
  'Colorado Rapids': {'GAM': -50000, 'TAM': 50000},
  'New England': {'GAM': 100000, 'TAM'

In [222]:
am_delta_records = pd.DataFrame({ 'year':[], 'club': [], 'gam_delta': [], 'tam_delta': []})
for year, data in am_table.items():
    for club, am in data.items():
        am_delta_records = am_delta_records.append({ 'year': year, 'club': club, 'gam_delta': am['GAM'], 'tam_delta': am['TAM'] }, ignore_index=True)
        
am_delta_records

,year,club,gam_delta,tam_delta
0,2018,New York City FC,0.0,0.0
1,2018,FC Cincinnati,-135000.0,-1050000.0
2,2018,New York Red Bulls,-225000.0,-150000.0
3,2018,Inter Miami,0.0,0.0
4,2018,Chicago,1125000.0,-200000.0
...,...,...,...,...
219,2025,Kansas City,0.0,0.0
220,2025,LA Galaxy,0.0,0.0
221,2025,Real Salt Lake,0.0,0.0
222,2025,Orlando City,0.0,0.0


In [223]:
am_delta_records.to_csv('./mls_am_delta.csv', index=False)

In [228]:
# https://www.sounderatheart.com/2021/2/8/22273209/details-of-revised-mls-collective-bargaining-agreement
# https://www.mlssoccer.com/news/mls-announces-increase-targeted-allocation-money-2018-and-2019
# https://www.brotherlygame.com/2019/12/20/21031618/allocation-money-explained
base_gam = { 
    '2018' : (1200000 + 150000), # non-discretionary + league allocation
    '2019' : (1200000 + 150000), # non-discretionary + league allocation
    '2020' : 1525000, 
    '2021': 1525000,
    '2022' : 1625000,
    '2023' : 1900000,
    '2024' : 2585000,
    '2025' : 2930000,
    '2026' : 3280000,
    '2027' : 3921000
}

base_tam = { 
    '2018' : 2800000, # discretionary
    '2019' : 2800000, # discretionary
    '2020' : 2800000, 
    '2021': 2800000,
    '2022' : 2800000,
    '2023' : 2720000,
    '2024' : 2400000,
    '2025' : 2225000,
    '2026' : 2125000,
    '2027' : 2025000
}

am_delta_records['base_tam'] = am_delta_records.year.apply(lambda x: base_gam[x])
am_delta_records['available_tam'] = am_delta_records['base_tam'] + am_delta_records['tam_delta']

am_delta_records['base_gam'] = am_delta_records.year.apply(lambda x: base_gam[x])
am_delta_records['available_gam'] = am_delta_records['base_gam'] + am_delta_records['gam_delta']

am_delta_records['available_xam'] = am_delta_records['available_gam'] + am_delta_records['available_tam'] 
am_delta_records

,year,club,gam_delta,tam_delta,base_tam,available_tam,base_gam,available_gam,available_xam
0,2018,New York City FC,0.0,0.0,1350000,1350000.0,1350000,1350000.0,2700000.0
1,2018,FC Cincinnati,-135000.0,-1050000.0,1350000,300000.0,1350000,1215000.0,1515000.0
2,2018,New York Red Bulls,-225000.0,-150000.0,1350000,1200000.0,1350000,1125000.0,2325000.0
3,2018,Inter Miami,0.0,0.0,1350000,1350000.0,1350000,1350000.0,2700000.0
4,2018,Chicago,1125000.0,-200000.0,1350000,1150000.0,1350000,2475000.0,3625000.0
...,...,...,...,...,...,...,...,...,...
219,2025,Kansas City,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0
220,2025,LA Galaxy,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0
221,2025,Real Salt Lake,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0
222,2025,Orlando City,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0


In [233]:
am_delta_records[am_delta_records.club == 'Toronto FC']

,year,club,gam_delta,tam_delta,base_tam,available_tam,base_gam,available_gam,available_xam
27,2018,Toronto FC,0.0,0.0,1350000,1350000.0,1350000,1350000.0,2700000.0
55,2019,Toronto FC,150000.0,50000.0,1350000,1400000.0,1350000,1500000.0,2900000.0
83,2020,Toronto FC,0.0,0.0,1525000,1525000.0,1525000,1525000.0,3050000.0
111,2021,Toronto FC,0.0,0.0,1525000,1525000.0,1525000,1525000.0,3050000.0
139,2022,Toronto FC,600000.0,0.0,1625000,1625000.0,1625000,2225000.0,3850000.0
167,2023,Toronto FC,0.0,0.0,1900000,1900000.0,1900000,1900000.0,3800000.0
195,2024,Toronto FC,0.0,0.0,2585000,2585000.0,2585000,2585000.0,5170000.0
223,2025,Toronto FC,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0


In [230]:
am_delta_records.to_csv('./base_mls_am_budgets.csv', index=False)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_am_budgets = pd.read_csv('./base_mls_am_budgets.csv')
base_am_budgets

,year,club,gam_delta,tam_delta,base_tam,available_tam,base_gam,available_gam,available_xam
0,2018,New York City FC,0.0,0.0,1350000,1350000.0,1350000,1350000.0,2700000.0
1,2018,FC Cincinnati,-135000.0,-1050000.0,1350000,300000.0,1350000,1215000.0,1515000.0
2,2018,New York Red Bulls,-225000.0,-150000.0,1350000,1200000.0,1350000,1125000.0,2325000.0
3,2018,Inter Miami,0.0,0.0,1350000,1350000.0,1350000,1350000.0,2700000.0
4,2018,Chicago,1125000.0,-200000.0,1350000,1150000.0,1350000,2475000.0,3625000.0
...,...,...,...,...,...,...,...,...,...
219,2025,Kansas City,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0
220,2025,LA Galaxy,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0
221,2025,Real Salt Lake,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0
222,2025,Orlando City,0.0,0.0,2930000,2930000.0,2930000,2930000.0,5860000.0


In [3]:
base_am_budgets.club.unique()

array(['New York City FC', 'FC Cincinnati', 'New York Red Bulls',
       'Inter Miami', 'Chicago', 'Charlotte FC', 'Austin FC', 'Houston',
       'FC Dallas', 'CF Montreal', 'Portland', 'LAFC', 'Atlanta United',
       'San Jose', 'Columbus Crew', 'Vancouver Whitecaps', 'Minnesota',
       'Seattle Sounders', 'Nashville', 'D.C. United', 'Colorado Rapids',
       'New England', 'Philadelphia', 'Kansas City', 'LA Galaxy',
       'Real Salt Lake', 'Orlando City', 'Toronto FC'], dtype=object)

In [5]:
# Resources
# Allocation Money disbursal amounts: https://en.wikipedia.org/wiki/Allocation_money
# 2020+ expansion GAM: https://www.charlottefootballclub.com/news/mls-101-allocation-money-salary-budgets-and-financial-rules
# csvs below built via wiki
allocation_csvs = ["ccl", "expansion", "expansion_draft", "leagues_cup","playoffs"]

base_additional_allocations = pd.DataFrame()
for item in allocation_csvs:
    file_path = f"./{item}_allocations.csv"
    tmp = pd.read_csv(file_path)
    base_additional_allocations = base_additional_allocations.append(tmp)
base_additional_allocations

,season,club,allocation_condition,gam_value
0,2018,Toronto FC,CCL,200000
1,2018,Kansas City,CCL,200000
2,2018,FC Dallas,CCL,200000
3,2018,Colorado Rapids,CCL,200000
4,2018,Seattle Sounders,CCL,200000
...,...,...,...,...
48,2022,LAFC,No Playoffs,200000
49,2022,San Jose,No Playoffs,200000
50,2022,FC Dallas,No Playoffs,200000
51,2022,Austin FC,No Playoffs,200000


In [6]:
base_additional_allocations.club.unique()

array(['Toronto FC', 'Kansas City', 'FC Dallas', 'Colorado Rapids',
       'Seattle Sounders', 'Atlanta United', 'New York Red Bulls',
       'Houston', 'LAFC', 'New York City FC', 'CF Montreal',
       'Philadelphia', 'Columbus Crew', 'New England', 'FC Cincinnati',
       'Nashville', 'Inter Miami', 'Austin FC', 'Charlotte FC',
       'San Jose', 'D.C. United', 'Vancouver Whitecaps', 'Minnesota',
       'Portland', 'LA Galaxy', 'Orlando City', 'Chicago',
       'Real Salt Lake', 'Minnesota United', 'Portland Timbers',
       'Colorado'], dtype=object)

In [8]:
list(set(base_additional_allocations.club.unique()) - set(base_am_budgets.club.unique()))

['Colorado', 'Minnesota United', 'Portland Timbers']

In [9]:
import numpy as np
base_additional_allocations.club = np.select(
    [
        (base_additional_allocations.club == 'Colorado'),
        (base_additional_allocations.club == 'Minnesota United'),
        (base_additional_allocations.club == 'Portland Timbers')
    ],
    [
        'Colorado Rapids',
        'Minnesota',
        'Portland'
    ]
    ,default = base_additional_allocations.club)
base_additional_allocations

,season,club,allocation_condition,gam_value
0,2018,Toronto FC,CCL,200000
1,2018,Kansas City,CCL,200000
2,2018,FC Dallas,CCL,200000
3,2018,Colorado Rapids,CCL,200000
4,2018,Seattle Sounders,CCL,200000
...,...,...,...,...
48,2022,LAFC,No Playoffs,200000
49,2022,San Jose,No Playoffs,200000
50,2022,FC Dallas,No Playoffs,200000
51,2022,Austin FC,No Playoffs,200000


In [10]:
list(set(base_additional_allocations.club.unique()) - set(base_am_budgets.club.unique()))

[]

In [11]:
base_additional_allocations.to_csv('./data/additional_allocations.csv', index=False)

In [12]:
# save state
asset_expl_flat = pd.read_csv('./allocation_money/mls_trade_db.csv')
asset_expl_flat

,trade_id,trade_date,club,acquired_asset_type,acquired_asset_value,acquired_asset_id,gam_value,asset_year,tam_value
0,0,2020-12-29,New York City FC,Player,$250000 2021 GAM,0,250000,2021,0
1,0,2020-12-29,New York City FC,Player,$250000 2022 GAM,1,250000,2022,0
2,0,2020-12-29,New York City FC,Player,$100000 2022 GAM (performance bonus),2,100000,2022,0
3,0,2020-12-29,FC Cincinnati,Player,Ronald Matarrita,3,0,2020,0
4,1,2020-12-23,New York Red Bulls,Player,$50000 2021 GAM,4,50000,2021,0
...,...,...,...,...,...,...,...,...,...
522,190,2019-01-11,Houston,Draft Pick,2019 2nd round pick (#33),522,0,2019,0
523,190,2019-01-11,D.C. United,Player,Chris Seitz (GK),523,0,2019,0
524,191,2019-01-11,Houston,Player,Marlon Hairston,524,0,2019,0
525,191,2019-01-11,Colorado Rapids,Player,$125000 2019 GAM,525,125000,2019,0
